# Convert HyperNeRF Data to EPIC-Diff Style

In [48]:
import numpy as np
from pathlib import Path
import json
import imageio
import shutil

np.set_printoptions(suppress=True)

epic_dir = Path('data/EPIC-Diff/')
# hypernerf_dir = Path('/home/tw554/Neural-Scene-Flow-Fields/data/')
hypernerf_dir = Path('data/hypernerf/')
scene_id = 'pick_drop'
# scene_id = 'kubric_single_car_rand_v2'
epic_data_dir = epic_dir / scene_id
epic_rgb_dir = epic_data_dir / 'frames'
epic_mask_dir = epic_data_dir / 'annotations'

epic_rgb_dir.mkdir(exist_ok=True, parents=True)
epic_mask_dir.mkdir(exist_ok=True, parents=True)

root_dir = hypernerf_dir / scene_id
img_scale = 4
# Where to save RGB images.
rgb_dir = root_dir / 'rgb' / f'{img_scale}x'
# Where to save mask images.
mask_dir = root_dir / 'mask' / f'{img_scale}x'
camera_dir = root_dir / 'camera-gt'
if not camera_dir.exists():
    camera_dir = root_dir / 'camera'

cam_list = sorted(list(camera_dir.glob('*')))

with open(root_dir / 'dataset.json','r') as f:
    dataset_data = json.load(f)
with open(root_dir / 'metadata.json','r') as f:
    time_data = json.load(f)

if (root_dir / 'scene_gt.json').exists():
    with open(root_dir / 'scene_gt.json','r') as f:
        scene_data = json.load(f)
else:
    with open(root_dir / 'scene.json','r') as f:
        scene_data = json.load(f)


rgb_list = [rgb_dir / f'{idx}.png' for idx in dataset_data['ids']]
if mask_dir.exists():
    mask_list = [mask_dir / f'{idx}.png' for idx in dataset_data['ids']]
else:
    mask_list = []

# filter nsff pose
nsff_pose = np.load(str(root_dir/ 'poses_bounds.npy'))
raw_rgb_lists = sorted(list(rgb_dir.glob('*')))
pose_ids = [raw_rgb_lists.index(i) for i in rgb_list]
nsff_pose = nsff_pose[pose_ids]

# mask_list = sorted(list(mask_dir.glob('*.png')))
# rgb_list = sorted(list(rgb_dir.glob('*.png')))

In [49]:
def normalize(x):
    return x / np.linalg.norm(x)

def viewmatrix(z, up, pos):
    vec2 = normalize(z)
    vec1_avg = up
    vec0 = normalize(np.cross(vec1_avg, vec2))
    vec1 = normalize(np.cross(vec2, vec0))
    m = np.stack([vec0, vec1, vec2, pos], 1)
    return m


def poses_avg(poses):

    hwf = poses[0, :3, -1:]

    center = poses[:, :3, 3].mean(0)
    vec2 = normalize(poses[:, :3, 2].sum(0))
    up = poses[:, :3, 1].sum(0)
    c2w = np.concatenate([viewmatrix(vec2, up, center), hwf], 1)
    
    return c2w

def recenter_poses(poses):

    poses_ = poses+0
    bottom = np.reshape([0,0,0,1.], [1,4])
    c2w = poses_avg(poses)
    c2w = np.concatenate([c2w[:3,:4], bottom], -2)
    bottom = np.tile(np.reshape(bottom, [1,1,4]), [poses.shape[0],1,1])
    poses = np.concatenate([poses[:,:3,:4], bottom], -2)

    poses = np.linalg.inv(c2w) @ poses
    poses_[:,:3,:4] = poses[:,:3,:4]
    poses = poses_
    return poses


poses = nsff_pose[:, :-2].copy().reshape([-1, 3, 5]).transpose([1,2,0])
bds = nsff_pose[:, -2:].transpose([1,0])

factor = img_scale

sh = imageio.imread(rgb_list[0]).shape
poses[:2, 4, :] = np.array(sh[:2]).reshape([2, 1])
poses[2, 4, :] = poses[2, 4, :] * 1./factor


poses = np.concatenate([poses[:, 1:2, :], 
                        -poses[:, 0:1, :], 
                        poses[:, 2:, :]], 1)
poses = np.moveaxis(poses, -1, 0).astype(np.float32)
bds = np.moveaxis(bds, -1, 0).astype(np.float32)

poses = recenter_poses(poses)

hwf = poses[0,:3,-1]
poses = poses[:,:3,:4]

In [53]:
poses.shape

(93, 3, 4)

In [50]:
# copy all the images and masks
for mask in mask_list:
    # shutil.copy(str(mask), str(mask_dir))
    img = imageio.imread(str(mask))
    imageio.imsave(str(epic_mask_dir / f'{mask.stem}.bmp'), img)
    

for rgb in rgb_list:
    # shutil.copy(str(rgb), str(rgb_dir))
    img = imageio.imread(str(rgb))
    imageio.imsave(str(epic_rgb_dir / f'{rgb.stem}.bmp'), img)

image_size = [img.shape[1], img.shape[0]]


In [54]:
# write meta
meta = {}

meta['ids_all'] = list(range(len(rgb_list)))
meta['ids_train'] = [dataset_data['ids'].index(idx) for idx in dataset_data['train_ids']]
# tes and val not used atm
meta['ids_val'] = [dataset_data['ids'].index(idx) for idx in dataset_data['val_ids']]
meta['ids_test'] = meta['ids_all']
# meta['ids_val'] = meta['ids_train']
# meta['ids_test'] = meta['ids_train']

# with open(cam_list[0],'r') as f:
#         cam_data = json.load(f)
intrinsics = np.zeros([3,3])
intrinsics[2,2] = 1
intrinsics[0,0] = intrinsics[1,1] = hwf[2]
intrinsics[0,2] = image_size[0] / 2
intrinsics[1,2] = image_size[1] / 2
meta['intrinsics'] = intrinsics.tolist()

meta['images'] = {}
meta['nears'] = {}
meta['fars'] = {}
meta['poses'] = {}

for i in range(len(rgb_list)):
    meta['images'][str(i)] = f'{rgb_list[i].stem}.bmp'
    meta['nears'][str(i)] = 0.
    meta['fars'][str(i)] = 1.

    rgb_list[i].stem

    c2w = poses[i]
    meta['poses'][str(i)] = c2w.tolist()

with open(epic_data_dir / 'meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

print(f"Num of imgs: {len(meta['ids_all'])}")
print(f"Num of train: {len(meta['ids_train'])}")
print(f"Num of val: {len(meta['ids_val'])}")
print(f"Num of test: {len(meta['ids_test'])}")



Num of imgs: 93
Num of train: 88
Num of val: 5
Num of test: 93
